version 1.0 

In [1]:
import numpy as np

# Constants
rho_0 = 1.225  # kg/m^3, atmospheric density at sea level
H = 8400.0  # m, scale height
G = 6.67430e-11  # m^3 kg^-1 s^-2, gravitational constant
M_earth = 5.972e24  # kg, Earth mass
R_earth = 6371000  # m, Earth radius
C_d = 2.2  # Drag coefficient
A = np.pi * (3.3 / 2)**2  # m^2, satellite windward area

def satellite_forces(position, velocity, velocity_direction):
    # Extract coordinates
    x, y = position
    vx, vy = velocity * np.cos(velocity_direction), velocity * np.sin(velocity_direction)

    # Atmospheric density calculation
    altitude = np.sqrt(x**2 + y**2) - R_earth
    rho = rho_0 * np.exp(-altitude / H)

    # Drag force calculation
    v = np.sqrt(vx**2 + vy**2)
    Fd = 0.5 * rho * v**2 * C_d * A
    Fd_x = Fd * (vx / v)
    Fd_y = Fd * (vy / v)

    # Gravitational force calculation
    r = np.sqrt(x**2 + y**2)
    Fg = G * M_earth / r**2
    Fg_x = -Fg * (x / r)
    Fg_y = -Fg * (y / r)

    # Total forces in x and y directions
    total_force_x = Fd_x + Fg_x
    total_force_y = Fd_y + Fg_y
    force_direction = np.arctan2(total_force_y, total_force_x)

    return np.sqrt(total_force_x**2 + total_force_y**2), force_direction

# Example usage
position = [0, 7000000]  # Example position (x=0, y=altitude+Earth radius)
velocity = 7500  # Example velocity in m/s
velocity_direction = 0  # Horizontal movement
force_magnitude, force_direction = satellite_forces(position, velocity, velocity_direction)
print("Force:", force_magnitude, "N")
print("Direction:", force_direction, "radians")


Force magnitude: 8.134473387755103 N
Force direction: -1.5707963267948966 radians


version 2.0(3D model with standerd atmospheric model)

In [15]:
import numpy as np
from datetime import datetime
from nrlmsise00 import msise_model

# Declaring Constants
G = 6.67430e-11  # Gravitational constant, m^3 kg^-1 s^-2
M_earth = 5.972e24  # Earth mass, kg
R_earth = 6371000  # Earth radius, m
C_d = 2.2  # Drag coefficient
m = 3000  # Satellite mass, kg
A = np.pi * (3.3 / 2)**2  # Satellite windward area, m^2

def satellite_forces(time, position, velocity, velocity_direction):
    # Convert velocity direction to three-dimensional velocity vector
    v_mag = velocity
    vx = v_mag * np.sin(velocity_direction[0]) * np.cos(velocity_direction[1])
    vy = v_mag * np.sin(velocity_direction[0]) * np.sin(velocity_direction[1])
    vz = v_mag * np.cos(velocity_direction[0])
    
    # Convert position vector to Earth center coordinates
    x, y, z = position
    r = np.sqrt(x**2 + y**2 + z**2)
    altitude = (r - R_earth) / 1000  # Convert units to kilometers

    # Get latitude and longitude of the current location
    lat = np.arcsin(z / r) * (180 / np.pi)  # Convert to degrees
    lon = np.arctan2(y, x) * (180 / np.pi)  # Convert to degrees
    
    # Obtain atmospheric density using the NRLMSISE-00 model
    output = msise_model(time, altitude, lat, lon, 150, 150, 4, lst=16)
    rho = output[0][5]  # Total mass density in kg/m^3

    # Calculate drag force components
    v = np.sqrt(vx**2 + vy**2 + vz**2)
    Fd = 0.5 * rho * v**2 * C_d * A
    Fd_x = Fd * (vx / v)
    Fd_y = Fd * (vy / v)
    Fd_z = Fd * (vz / v)

    # Calculate gravitational force components
    Fg = G * M_earth / r**2
    Fg_x = -Fg * (x / r)
    Fg_y = -Fg * (y / r)
    Fg_z = -Fg * (z / r)

    # Total force components
    total_force_x = Fd_x + Fg_x
    total_force_y = Fd_y + Fg_y
    total_force_z = Fd_z + Fg_z
    force_magnitude = np.sqrt(total_force_x**2 + total_force_y**2 + total_force_z**2)

    # Calculate force direction in spherical coordinates
    theta = np.arccos(total_force_z / force_magnitude)  # Elevation angle
    phi = np.arctan2(total_force_y, total_force_x)  # Azimuth angle

    return force_magnitude, (theta, phi)  # Return both magnitude and direction tuple

# Example usage
time = datetime(2009, 6, 21, 8, 3, 20)
position = [0, 0, 7071000]  # Three-dimensional position (including Earth radius)
velocity = 7500  # velocity, m/s
velocity_direction = [np.pi / 2, 0]  # Velocity direction, spherical coordinate form [theta, phi]
force_magnitude, force_direction= satellite_forces(time, position, velocity, velocity_direction)
print("Force:", force_magnitude, "N")
print("Direction:", force_direction, "theta, phi")

force: 7.971936821748241 N
direction: (3.141592653589793, 0.0) theta, phi
